In [1]:
!python -m spacy download en_core_web_lg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.7/587.7 MB 2.8 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


In [2]:
!pip install transformers==4.39.0
!spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 1.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 35.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 77.5 MB/s eta 0:00:00:00:01
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully uninstalled tokenizers-0.19.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.42.3
    Uninstalling transformers-4.42.3:
      Successfully uninstalled transformers-4.42.3
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 81.7 MB/s eta 0:00:0000:010:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [3]:
!python -m pip install -U "setfit[absa]"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 733.4 kB/s eta 0:00:000:00:010m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 3.5 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 4.8 MB/s eta 0:00:00


In [ ]:
## from datasets import Dataset
from setfit import AbsaModel, AbsaTrainer,TrainingArguments
import pandas as pd
from datasets import Dataset, load_dataset
from transformers import EarlyStoppingCallback
# Excel dosyasını okuyun
df = pd.read_excel('/kaggle/input/dataaa/donusmus_son_hal.xlsx')


# Veriyi datasets formatına dönüştür
dataset = Dataset.from_pandas(df)

# Veri setini eğitim ve geçerli test seti olarak ayır
train_val_split = dataset.train_test_split(test_size=0.2, shuffle=True, seed=42)

train_dataset = train_val_split['train']

val_test_split = train_val_split['test'].train_test_split(test_size=0.5, shuffle=True, seed=42)

# Doğrulama ve test setlerini alın
val_dataset = val_test_split['train']
test_dataset = val_test_split['test']

model = AbsaModel.from_pretrained(
    "sentence-transformers/all-MiniLM-L6-v2",
    spacy_model="en_core_web_sm",
)

args = TrainingArguments(
    output_dir="models",
    num_epochs=1,
    batch_size=64,
    evaluation_strategy="steps",
    eval_steps=10000,
    save_steps=10000,
    load_best_model_at_end=True,
)

trainer = AbsaTrainer(
    model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=5)],
)
trainer.train()

/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.
model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.
/opt/conda/lib/python3.10/site-packages/ipywidgets/widgets/widget.py:503: DeprecationWarning: The `ipykernel.comm.Comm` class has been deprecated. Please use the `comm` module instead.For creating comms, use the function `from comm import create_comm`.
  self.comm = Comm(**args)


Map:   0%|          | 0/2664 [00:00<?, ? examples/s]

Map:   0%|          | 0/578 [00:00<?, ? examples/s]

***** Running training *****
  Num unique pairs = 4688144
  Batch size = 64
  Num epochs = 1
  Total optimization steps = 73253


Step,Training Loss,Validation Loss,Embedding Loss,Rate
10000,No log,No log,0.050800,0.000019
20000,No log,No log,0.067300,0.000016


  0%|          | 0/1079 [00:00<?, ?it/s]

  0%|          | 0/1079 [00:00<?, ?it/s]

In [ ]:
metrics = trainer.evaluate(test_dataset)
print(metrics)

In [ ]:
model.save_pretrained(
    "/kaggle/working/yarısma-aspect",
    "/kaggle/working/yarısma-polarity",
)

In [ ]:
from setfit import AbsaModel

model = AbsaModel.from_pretrained(
    "/kaggle/working/yarısma-aspect",
    "/kaggle/working/yarısma-polarity",
    spacy_model="en_core_web_sm",
)

In [ ]:
preds = model.predict([
    "la roche çok güzel ve yüzümdeki sivilce lekelerinin azaldığını farkettim kesinlikle stok yapılmalı",
    "Casio saati çok sağlam kızım için aldım çok memnun.",
    "Unfortunately, you have to expect some waiting time and get a note with a waiting number if it should be very full."
])
print(preds)

In [ ]:
# Mevcut model tahminleri
preds = model.predict([
    "Trendyol çok iyidir",
    "Unfortunately, you have to expect some waiting time and get a note with a waiting number if it should be very full."
])

# Tahminlerin yeni formata dönüştürülmesi
result = {
    "entity_list": [],
    "results": []
}

# Tahmin edilen verilerin işlenmesi
for pred in preds:
    for item in pred:
        entity = item['span']
        sentiment = item['polarity']
        result["results"].append({
            "entity": entity,
            "sentiment": sentiment
        })
        result["entity_list"].append(entity)


# Sonucu yazdırma
print(result)